# Problem Statement

Code pro is an edtech startup that had a phenomenal seed A funding round. They used this funding to gain significant market share by campaigning aggressively. Due to aggressive campaigns the amount of junk leads increased.

A lead is generated when any person visits Code pro’s website and enters their contact details on the platform. A junk lead is when the person sharing his contact details has no interest in the product/service.

Having junk leads in the pipeline creates a lot of inefficiency in the sales process. Thus the goal of the Data Science team is to build a system that categorizes leads on their propensity to purchase Code pros course. This system will help remove the inefficiency caused by junk leads in the sales process.

# 1.Understanding the data

Inorder to classify any lead we will require 2 types of information about the lead. The 2 types are

1.Source of origin of the lead:  In order to better predict the propensity of the lead to purchase the course, we need some information on the source of origin of the lead. The column from ‘city_mapped’ to ‘reffered_leads’ contain the information about the source of origin of the lead

2.Interaction of the lead with the website/platform: In order to better predict the propensity of the lead to purchase the course, we need some information on what was the interaction of the lead with the platform. The column from ‘1_on_1_industry_mentorship’ to ‘whatsapp_chat_click’ store the information about the interaction the lead had with the platform.

In [21]:
# ignore the warnings in the output
import warnings
warnings.filterwarnings("ignore")

In [22]:
# import python packages 

import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

In [23]:
# read the leadscoring data
df_lead_scoring = pd.read_csv('Data/leadscoring.csv', index_col=[0])

In [24]:
# lead scoring data sample
df_lead_scoring.head()

,created_date,city_mapped,first_platform_c,first_utm_medium_c,first_utm_source_c,total_leads_droppped,referred_lead,1_on_1_industry_mentorship,call_us_button_clicked,career_assistance,...,specilization_click,syllabus,syllabus_expand,syllabus_submodule_expand,tab_career_assistance,tab_job_opportunities,tab_student_support,view_programs_page,whatsapp_chat_click,app_complete_flag
0,2021-08-13 10:44:09,mumbai,Level0,Level0,Level0,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2021-11-04 06:34:10,ncr,Level1,Level1,Level1,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2021-07-29 14:01:00,meerut,Level0,Level2,Level2,2.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,2021-12-01 13:21:54,ncr,Level1,Level3,Level3,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,2021-12-10 07:22:51,ahmedabad,Level2,Level2,Level2,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


# 2. Exploratory Data Analysis

For understanding the source of origin of the lead we have the following information

1.created_data: Timestamp at which the lead was created <br>
2.city_mapped: The city from which the lead was generated <br>
3.first_platform_c: Descirbes the platform from which the lead was generated. Examples include mobile web, Desktop, Android app etc. In our case for privacy we have mapped these to levels. <br>
4.first_utm_medium_c:  Mediums describe the type of traffic. It can be of various types a few of which are 'Organic', 'Referral', 'Paid' etc.<br>
5.first_utm_source_c: Source is where your website’s traffic comes from (individual websites, Google, Facebook etc).<br>
6.total_leads_droppped: Total number of courses a lead checked<br>
7.reffered_lead: Whether the lead was reffered our not<br>
8.1_on_1_industry_mentorship: If the user has clicked on 1-on-1 industry mentorship<br>
9.call_us_button_clicked: If the user has clicked on call us button<br>
...<br>
44.whatsapp_chat_click : If the user has clicked on whatsapp chat<br>
45.app_complete_flag: this is the column we are trying to predict. We are trying to understand whether a user will fill the application or not based on the information in the previous columns<br>



In [25]:
profile = ProfileReport(df_lead_scoring, title="Raw Data Summary")
#profile.to_notebook_iframe()
profile.to_file('profile_report/raw_data_report.html')

Summarize dataset:   0%|          | 0/58 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Since there are a lot of unique values of cities, we will use the tier of the cities instead.

 # 3. Data Preprocessing

In [26]:
from Maps.city_tier import city_tier_mapping

In [27]:
df_lead_scoring["city_tier"] = df_lead_scoring["city_mapped"].map(city_tier_mapping)
df_lead_scoring["city_tier"] = df_lead_scoring["city_tier"].fillna(3.0)

In [28]:
df_lead_scoring.head()

,created_date,city_mapped,first_platform_c,first_utm_medium_c,first_utm_source_c,total_leads_droppped,referred_lead,1_on_1_industry_mentorship,call_us_button_clicked,career_assistance,...,syllabus,syllabus_expand,syllabus_submodule_expand,tab_career_assistance,tab_job_opportunities,tab_student_support,view_programs_page,whatsapp_chat_click,app_complete_flag,city_tier
0,2021-08-13 10:44:09,mumbai,Level0,Level0,Level0,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0
1,2021-11-04 06:34:10,ncr,Level1,Level1,Level1,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0
2,2021-07-29 14:01:00,meerut,Level0,Level2,Level2,2.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2.0
3,2021-12-01 13:21:54,ncr,Level1,Level3,Level3,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0
4,2021-12-10 07:22:51,ahmedabad,Level2,Level2,Level2,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2.0


In [29]:
# calculate the frequency of individual category in first_platform_c column
df_cat_freq = df_lead_scoring['first_platform_c'].value_counts() 
df_cat_freq

Level0     84532
Level3     41028
Level7     38267
Level1     20832
Level2     13284
Level8     10838
Level5      9768
Level4      5514
Level16     3790
Level10     3288
Level12     3153
Level18     2130
Level14      751
Level11      714
Level6       662
Level13      554
Level19      258
Level15       98
Level20       75
Level22       65
Level26       65
Level17       53
Level23       42
Level27       30
Level33       29
Level21       25
Level30       21
Level39       18
Level24       17
Level35       16
Level31       11
Level41        9
Level32        9
Level37        9
Level25        8
Level36        8
Level34        7
Level9         5
Level29        4
Level47        3
Level38        2
Level28        1
Level40        1
Level42        1
Level43        1
Level44        1
Level45        1
Level46        1
Level48        1
Name: first_platform_c, dtype: int64

In [30]:
# index the dataframe that we created above 
df_cat_freq = pd.DataFrame({'column':df_cat_freq.index, 'value':df_cat_freq.values})
df_cat_freq

,column,value
0,Level0,84532
1,Level3,41028
2,Level7,38267
3,Level1,20832
4,Level2,13284
5,Level8,10838
6,Level5,9768
7,Level4,5514
8,Level16,3790
9,Level10,3288


In [31]:
# calculate the cumulative percentage for each category. This is given by calculating cumulative sum and dividing by total sum for each category.
df_cat_freq['perc'] = df_cat_freq['value'].cumsum()/df_cat_freq['value'].sum()
df_cat_freq

,column,value,perc
0,Level0,84532,0.352217
1,Level3,41028,0.523167
2,Level7,38267,0.682612
3,Level1,20832,0.769413
4,Level2,13284,0.824762
5,Level8,10838,0.869921
6,Level5,9768,0.910621
7,Level4,5514,0.933596
8,Level16,3790,0.949388
9,Level10,3288,0.963087


<b> As seen in the pandas profiling report, we have leads originating from various utm medium and sources, but majority of the leads come from only a few of these sources and mediums. In order to overcome this we will group all the insignificant sources and medium into 'others'. Doing so will prevent information loss </b>

In [32]:
# cumulative percetage is calculated for all the three columns - first_platform_c, first_utm_medium_c, first_utm_source_c
for column in ['first_platform_c', 'first_utm_medium_c', 'first_utm_source_c']:
    df_cat_freq = df_lead_scoring[column].value_counts()
    df_cat_freq = pd.DataFrame({'column':df_cat_freq.index, 'value':df_cat_freq.values})
    df_cat_freq['perc'] = df_cat_freq['value'].cumsum()/df_cat_freq['value'].sum()
    print(column,)
    print(list(df_cat_freq.loc[df_cat_freq['perc']<=0.9].column))
    print('\n')

first_platform_c
['Level0', 'Level3', 'Level7', 'Level1', 'Level2', 'Level8']


first_utm_medium_c
['Level0', 'Level2', 'Level6', 'Level3', 'Level4', 'Level9', 'Level11', 'Level5', 'Level8', 'Level20', 'Level13', 'Level30', 'Level33', 'Level16', 'Level10', 'Level15', 'Level26', 'Level43']


first_utm_source_c
['Level2', 'Level0', 'Level7', 'Level4', 'Level6', 'Level16', 'Level5', 'Level14']




In [33]:
# levels are stored in list for further processing
list_platform=['Level0', 'Level3', 'Level7', 'Level1', 'Level2', 'Level8']

list_medium=['Level0', 'Level2', 'Level6', 'Level3', 'Level4', 'Level9', 'Level11', 'Level5', 'Level8', 'Level20', 'Level13', 'Level30', 'Level33', 'Level16', 'Level10', 'Level15', 'Level26', 'Level43']

list_source=['Level2', 'Level0', 'Level7', 'Level4', 'Level6', 'Level16', 'Level5', 'Level14']

In [34]:
# all the levels below 90 percentage are assgined to a single level called others
new_df = df_lead_scoring[~df_lead_scoring['first_platform_c'].isin(list_platform)] # get rows for levels which are not present in list_platform
new_df['first_platform_c'] = "others" # replace the value of these levels to others
old_df = df_lead_scoring[df_lead_scoring['first_platform_c'].isin(list_platform)] # get rows for levels which are present in list_platform
df = pd.concat([new_df, old_df]) # concatenate new_df and old_df to get the final dataframe

In [35]:
# all the levels below 90 percentage are assgined to a single level called others
new_df = df[~df['first_utm_medium_c'].isin(list_medium)] # get rows for levels which are not present in list_medium
new_df['first_utm_medium_c'] = "others" # replace the value of these levels to others
old_df = df[df['first_utm_medium_c'].isin(list_medium)] # get rows for levels which are present in list_medium
df = pd.concat([new_df, old_df]) # concatenate new_df and old_df to get the final dataframe

In [36]:
# all the levels below 90 percentage are assgined to a single level called others
new_df = df[~df['first_utm_source_c'].isin(list_source)] # get rows for levels which are not present in list_source
new_df['first_utm_source_c'] = "others" # replace the value of these levels to others
old_df = df[df['first_utm_source_c'].isin(list_source)] # get rows for levels which are present in list_source
df = pd.concat([new_df, old_df]) # concatenate new_df and old_df to get the final dataframe

In [37]:
df.head()

,created_date,city_mapped,first_platform_c,first_utm_medium_c,first_utm_source_c,total_leads_droppped,referred_lead,1_on_1_industry_mentorship,call_us_button_clicked,career_assistance,...,syllabus,syllabus_expand,syllabus_submodule_expand,tab_career_assistance,tab_job_opportunities,tab_student_support,view_programs_page,whatsapp_chat_click,app_complete_flag,city_tier
1528,2021-09-13 07:49:28,nagpur,others,others,others,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2.0
1618,2021-11-01 08:49:05,vizayanagara,others,others,others,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3.0
2970,2021-08-02 06:26:55,ncr,others,others,others,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0
3077,2021-11-22 07:38:38,pune,others,others,others,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0
3479,2021-11-01 08:44:51,yavatmal,others,others,others,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3.0


In [38]:
df.isnull().sum()

created_date                                                     0
city_mapped                                                   9403
first_platform_c                                                 0
first_utm_medium_c                                               0
first_utm_source_c                                               0
total_leads_droppped                                           826
referred_lead                                                  826
1_on_1_industry_mentorship                                  240000
call_us_button_clicked                                      239998
career_assistance                                           239996
career_coach                                                240000
career_impact                                               239995
careers                                                     239980
chat_clicked                                                240000
companies                                                   23

In [39]:
df['total_leads_droppped'] = df['total_leads_droppped'].fillna(0)
df['referred_lead'] = df['referred_lead'].fillna(0)

In [40]:
df = df.drop(['city_mapped'], axis = 1)

# Interactions with the platform



Recall that our sales team requested information about the type of interaction that the lead had with the platform. Thus we will combine all the interaction columns into 4 broad categories namely

1. Assitance Interaction: This counts the number of times a lead has interacted with the assistance options available on the platform. 
2. Career Interaction: This counts the number of times a lead has interacted with the career pages on the platform. This includes the number of times the lead checked out the placement of the Upgrads alumnis, the companies where they are placed etc.
3. Payment Interaction: This counts the number of times a lead has interacted with the payment plans on the platform. This included the number of times the lead checked out the EMI options, visited EMI partners etc on the platform
4. Syllabus Interaction: This counts the number of times a lead has interacted with the syllabus of the course. This included the number of 

Categorisng the interactions in these 4 broad ways will imporve the intrepretibility and will reduce the sparsity of the iteractions data. This mapping of the 100 or so events into Assistance, Career, Payment and Syllabus will also help the sales team customise their pitch.

In [41]:
df = df.drop_duplicates()

In [42]:
# read the interaction mapping file
df_event_mapping = pd.read_csv('Maps/interaction_mapping.csv', index_col=[0])

In [43]:
df.shape

(238964, 45)

In [44]:
df.loc[(df['created_date'] == '2021-11-04 06:34:10')]

,created_date,first_platform_c,first_utm_medium_c,first_utm_source_c,total_leads_droppped,referred_lead,1_on_1_industry_mentorship,call_us_button_clicked,career_assistance,career_coach,...,syllabus,syllabus_expand,syllabus_submodule_expand,tab_career_assistance,tab_job_opportunities,tab_student_support,view_programs_page,whatsapp_chat_click,app_complete_flag,city_tier
1,2021-11-04 06:34:10,Level1,others,others,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0


In [45]:
# unpivot the interaction columns and put the values in rows
df_unpivot = pd.melt(df, id_vars=['created_date', 'first_platform_c',
       'first_utm_medium_c', 'first_utm_source_c', 'total_leads_droppped', 'city_tier',
       'referred_lead', 'app_complete_flag'], var_name='interaction_type', value_name='interaction_value')

In [46]:
# 37 interaction columns, each row gets converted into 37 rows 
df_unpivot.shape

(8841668, 10)

In [47]:
# example of single row converted into 37 rows
df_unpivot.loc[(df_unpivot['created_date'] == '2021-11-04 06:34:10')]

,created_date,first_platform_c,first_utm_medium_c,first_utm_source_c,total_leads_droppped,city_tier,referred_lead,app_complete_flag,interaction_type,interaction_value
395,2021-11-04 06:34:10,Level1,others,others,1.0,1.0,0.0,0,1_on_1_industry_mentorship,NaN
239359,2021-11-04 06:34:10,Level1,others,others,1.0,1.0,0.0,0,call_us_button_clicked,NaN
478323,2021-11-04 06:34:10,Level1,others,others,1.0,1.0,0.0,0,career_assistance,NaN
717287,2021-11-04 06:34:10,Level1,others,others,1.0,1.0,0.0,0,career_coach,NaN
956251,2021-11-04 06:34:10,Level1,others,others,1.0,1.0,0.0,0,career_impact,NaN
1195215,2021-11-04 06:34:10,Level1,others,others,1.0,1.0,0.0,0,careers,NaN
1434179,2021-11-04 06:34:10,Level1,others,others,1.0,1.0,0.0,0,chat_clicked,NaN
1673143,2021-11-04 06:34:10,Level1,others,others,1.0,1.0,0.0,0,companies,NaN
1912107,2021-11-04 06:34:10,Level1,others,others,1.0,1.0,0.0,0,download_button_clicked,NaN
2151071,2021-11-04 06:34:10,Level1,others,others,1.0,1.0,0.0,0,download_syllabus,NaN


In [48]:
# handle the nulls in the interaction value column
df_unpivot['interaction_value'] = df_unpivot['interaction_value'].fillna(0)

In [49]:
# map interaction type column with the mapping file to get interaction mapping
df = pd.merge(df_unpivot, df_event_mapping, on='interaction_type', how='left')

In [50]:
df.head()

,created_date,first_platform_c,first_utm_medium_c,first_utm_source_c,total_leads_droppped,city_tier,referred_lead,app_complete_flag,interaction_type,interaction_value,interaction_mapping
0,2021-09-13 07:49:28,others,others,others,1.0,2.0,0.0,0,1_on_1_industry_mentorship,0.0,career_interaction
1,2021-11-01 08:49:05,others,others,others,1.0,3.0,0.0,0,1_on_1_industry_mentorship,0.0,career_interaction
2,2021-08-02 06:26:55,others,others,others,1.0,1.0,0.0,0,1_on_1_industry_mentorship,0.0,career_interaction
3,2021-11-22 07:38:38,others,others,others,1.0,1.0,0.0,0,1_on_1_industry_mentorship,0.0,career_interaction
4,2021-11-01 08:44:51,others,others,others,1.0,3.0,0.0,0,1_on_1_industry_mentorship,0.0,career_interaction


In [51]:
df.shape

(8841668, 11)

In [52]:
#dropping the interaction type column as it is not needed
df = df.drop(['interaction_type'], axis=1)

In [53]:
# interaction type column has been removed and replaced by interaction mapping column
df.loc[(df['created_date'] == '2021-11-04 06:34:10')]

,created_date,first_platform_c,first_utm_medium_c,first_utm_source_c,total_leads_droppped,city_tier,referred_lead,app_complete_flag,interaction_value,interaction_mapping
395,2021-11-04 06:34:10,Level1,others,others,1.0,1.0,0.0,0,0.0,career_interaction
239359,2021-11-04 06:34:10,Level1,others,others,1.0,1.0,0.0,0,0.0,assistance_interaction
478323,2021-11-04 06:34:10,Level1,others,others,1.0,1.0,0.0,0,0.0,career_interaction
717287,2021-11-04 06:34:10,Level1,others,others,1.0,1.0,0.0,0,0.0,career_interaction
956251,2021-11-04 06:34:10,Level1,others,others,1.0,1.0,0.0,0,0.0,career_interaction
1195215,2021-11-04 06:34:10,Level1,others,others,1.0,1.0,0.0,0,0.0,career_interaction
1434179,2021-11-04 06:34:10,Level1,others,others,1.0,1.0,0.0,0,0.0,assistance_interaction
1673143,2021-11-04 06:34:10,Level1,others,others,1.0,1.0,0.0,0,0.0,assistance_interaction
1912107,2021-11-04 06:34:10,Level1,others,others,1.0,1.0,0.0,0,0.0,syllabus_interaction
2151071,2021-11-04 06:34:10,Level1,others,others,1.0,1.0,0.0,0,0.0,syllabus_interaction


In [54]:
# pivoting the interaction mapping column values to individual columns in the dataset
df_pivot = df.pivot_table(
        values='interaction_value', index=['created_date', 'city_tier', 'first_platform_c',
       'first_utm_medium_c', 'first_utm_source_c', 'total_leads_droppped',
       'referred_lead', 'app_complete_flag'], columns='interaction_mapping', aggfunc='sum')
df_pivot = df_pivot.reset_index()

In [55]:
# the rows are again converted back to columns. 37 rows gets converted to a single row.
df_pivot.shape

(238964, 13)

In [56]:
df_pivot.head()

interaction_mapping,created_date,city_tier,first_platform_c,first_utm_medium_c,first_utm_source_c,total_leads_droppped,referred_lead,app_complete_flag,assistance_interaction,career_interaction,payment_interaction,social_interaction,syllabus_interaction
0,2021-07-01 00:08:15,1.0,Level0,Level11,Level2,1.0,0.0,1,0.0,0.0,0.0,0.0,0.0
1,2021-07-01 00:16:43,2.0,Level3,Level0,others,1.0,0.0,1,0.0,0.0,0.0,0.0,0.0
2,2021-07-01 00:22:20,1.0,Level3,Level0,Level0,1.0,0.0,1,0.0,0.0,0.0,0.0,0.0
3,2021-07-01 00:23:13,1.0,Level1,Level3,others,2.0,0.0,0,0.0,0.0,0.0,0.0,0.0
4,2021-07-01 00:28:38,1.0,Level3,Level0,Level0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0


In [57]:
# the above rows have been reduced back to a single row
df_pivot.loc[(df['created_date'] == '2021-11-04 06:34:10')]

interaction_mapping,created_date,city_tier,first_platform_c,first_utm_medium_c,first_utm_source_c,total_leads_droppped,referred_lead,app_complete_flag,assistance_interaction,career_interaction,payment_interaction,social_interaction,syllabus_interaction
395,2021-07-01 10:10:02,2.0,Level0,Level2,Level2,1.0,0.0,1,0.0,0.0,0.0,0.0,0.0


In [58]:
# generate the profile report of final dataset

profile = ProfileReport(df_pivot, title="Cleaned Data Summary")
profile.to_notebook_iframe()
profile.to_file('profile_report/cleaned_data_report.html')

Summarize dataset:   0%|          | 0/26 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [59]:
# the file is written in the data folder
df_pivot.to_csv('Data/cleaned_data.csv', index=False)